# Luciano Ramalho - Pythonic Objects: idiomatic OOP in Python - PyCon 2019

**Link to talk:** https://www.youtube.com/watch?v=mUu_4k6a5-I

**Link to talk repo:** https://github.com/ramalho/pyob

_"The determined Real Programmer can write FORTRAN programs in any language."_ - Ed Post, Real Programmers Don't Use Pascal, 1982.

### Quotes from the talk

* Python presents itself to the programmer as a toolkit for building frameworks.

# **Part 1: Attribute Basics**

In [1]:
# Check version
import sys
print(sys.version)

3.7.3 | packaged by conda-forge | (default, Mar 27 2019, 23:01:00) 
[GCC 7.3.0]


### A simplistic class

In [2]:
class Coordinate:
    '''Coordinate on Earth'''

In [3]:
# Assigning new class attributes at will is permitted, but not recommended
cle = Coordinate()
cle.lat = 41.4
cle.long = -81.1
cle

In [4]:
cle.lat

41.4

### First method: `__repr__`

In [5]:
class Coordinate:
    '''Coordinate on Earth'''
    
    def __repr__(self):
        return f'Coordinate({self.lat}, {self.long})'

In [6]:
cle = Coordinate()
cle.lat = 41.1
cle.long = -81.8
cle

Coordinate(41.1, -81.8)

### About `__repr__`

* Good for exploratory programming, docs, doctests, and debugging
* Best practice: if viable, make `__repr__` return string with syntax required to make a new instance (like example above)
* If not viable, use `<MyClass ...>`  with some `...` that identifies the particular instance

In [7]:
cle.__repr__()

'Coordinate(41.1, -81.8)'

In [8]:
# This syntax is a builtin special method, faster than above
repr(cle)

'Coordinate(41.1, -81.8)'

### `__str__` is for end-user displays

In [9]:
class Coordinate:
    '''Coordinate on Earth'''
    
    def __repr__(self):
        return f'Coordinate({self.lat}, {self.long})'
    
    def __str__(self):
        # bool is a subtype of int
        # so this bool expr indexes into the str
        ns = 'NS'[self.lat < 0]
        we = 'EW'[self.long < 0]
        return f'{abs(self.lat):.1f}°{ns}, {abs(self.long):.1f}°{we}'

In [10]:
cle = Coordinate()
cle.lat = 41.1
cle.long = -81.8
cle

Coordinate(41.1, -81.8)

In [11]:
# print calls __str__(self)
print(cle)

41.1°N, 81.8°W


### But...

Printing the `Coordinate` object fails because the `lat` and `long` attributes don't exist.

In [12]:
gulf_of_guinea = Coordinate()
try:
    print(gulf_of_guinea)
except AttributeError as e:
    print(e)

'Coordinate' object has no attribute 'lat'


### So...

Redefine `Coordinate` to have default values for the necessary class attributes. This is not good practice, though.

In [13]:
class Coordinate:
    '''Coordinate on Earth'''
    
    lat = 0.0
    long = 0.0

    def __repr__(self):
        return f'Coordinate({self.lat}, {self.long})'
    
    def __str__(self):
        # bool is a subtype of int
        # so this bool expr indexes into the str
        ns = 'NS'[self.lat < 0]
        we = 'EW'[self.long < 0]
        return f'{abs(self.lat):.1f}°{ns}, {abs(self.long):.1f}°{we}'

In [14]:
gulf_of_guinea = Coordinate()
try:
    print(gulf_of_guinea)
except AttributeError as e:
    print(e)

0.0°N, 0.0°E


### `__dict__` is a dictionary of the attributes of a particular instance.

In [15]:
class Pizza:
    
    diameter = 40 # cm
    slices = 8
    
    flavor = 'cheese'
    flavor2 = None

In [16]:
p = Pizza()
p.slices

8

In [17]:
p.flavor

'cheese'

The two cells below show that `__dict__` doesn't look up values in the class definition. It only looks at values for the _instance_.

In [18]:
p.__dict__

{}

In [19]:
p.flavor = 'Sausage'
p.__dict__

{'flavor': 'Sausage'}

Using `__dict__` on the class.

In [20]:
Pizza.__dict__

mappingproxy({'__module__': '__main__',
              'diameter': 40,
              'slices': 8,
              'flavor': 'cheese',
              'flavor2': None,
              '__dict__': <attribute '__dict__' of 'Pizza' objects>,
              '__weakref__': <attribute '__weakref__' of 'Pizza' objects>,
              '__doc__': None})

### A better pizza...

Best practices:

* Use of _class attributes_ for attributes shared by all instances
* Attributes that are expected to vary among instances are _instance attributes_
* Assign all _instance attributes_ in `__init__` 
* Default values for instance attributes are `__init__` argument defualts

In [21]:
class Pizza:
    
    diameter = 40 # cm
    slices = 8
    
    def __init__(self, flavor='cheese', flavor2=None):
        self.flavor = flavor
        self.flavor2 = flavor2

# Lab 1: enhancing `Coordinate`

Follow instructions from: https://github.com/ramalho/pyob/tree/master/labs/1

**Step 1:** Implement `__init__` taking `lat` and `long`, both optional with 0.0 as default.

**Step 2:** Create a class attribute named `reference_system` with value 'WGS84'.

**Step 3:** Use the encode function of the geohash.py module to create a `geohash` method in the Coordinate class.

In [22]:
import geohash

class Coordinate:
    '''Coordinate on Earth'''

    reference_system = 'WGS84'
    
    
    def __init__(self, lat=0.0, long=0.0):
        self.lat = lat
        self.long = long

    def __repr__(self):
        return f'Coordinate({self.lat}, {self.long})'
    
    def __str__(self):
        # bool is a subtype of int
        # so this bool expr indexes into the str
        ns = 'NS'[self.lat < 0]
        we = 'EW'[self.long < 0]
        return f'{abs(self.lat):.1f}°{ns}, {abs(self.long):.1f}°{we}'
    
    def geohash(self):
        return geohash.encode(self.lat, self.long)

In [23]:
london = Coordinate(-1.1, 23.4)
print(london)
print(london.geohash())
london

1.1°S, 23.4°E
kxb9srz2590s


Coordinate(-1.1, 23.4)

------------

# **Part 2: Caveats with mutable attributes and arguments**

In [24]:
# A simple class to illustrate the danger of a mutable class attribute used as a
# default value for an instance attribute
class HauntedBus:
    '''A bus haunted by ghost passengers'''
    
    passengers = []
    
    def pick(self, name):
        self.passengers.append(name)
        
    def drop(self, name):
        self.passengers.remove(name)

In [25]:
bus1 = HauntedBus()
bus1.passengers

[]

In [26]:
bus1.pick('Ann')
bus1.pick('Bob')
bus1.passengers

['Ann', 'Bob']

In [27]:
# Ghost passengers!
bus2 = HauntedBus()
bus2.passengers

['Ann', 'Bob']

In [28]:
class HauntedBus_v2:
    '''Another bus haunted by ghost passengers'''
    
    def __init__(self, passengers=[]):
        self.passengers = passengers
    
    def pick(self, name):
        self.passengers.append(name)
        
    def drop(self, name):
        self.passengers.remove(name)

In [29]:
bus3 = HauntedBus_v2()
bus3.passengers

[]

In [30]:
bus3.pick('Charlie')
bus3.pick('Debbie')
bus3.passengers

['Charlie', 'Debbie']

In [31]:
# Ghost passengers!
bus4 = HauntedBus_v2()
bus4.passengers

['Charlie', 'Debbie']

The `pick` and `drop` methods were changing the default value for the passengers argument in the `__init__` method. **The argument defaults are also class attributes** (indirectly because `__init__` is a class attribute).

In [32]:
HauntedBus_v2.__init__.__defaults__

(['Charlie', 'Debbie'],)

In [33]:
class TwilightBus:
    '''A bus model that makes passengeers vanish'''
    
    def __init__(self, passengers=None):
        if passengers is None:
            self.passengers = []
        else:
            self.passengers = passengers
            
    def pick(self, name):
        self.passengers.append(name)
        
    def drop(self, name):
        self.passengers.remove(name)

In [34]:
hockey_team = ['Sue', 'Tina', 'Maya', 'Diana', 'Pat', 'Alice']
bus5 = TwilightBus(hockey_team)
bus5.passengers

['Sue', 'Tina', 'Maya', 'Diana', 'Pat', 'Alice']

In [35]:
bus5.drop('Sue')
bus5.drop('Pat')
bus5.passengers

['Tina', 'Maya', 'Diana', 'Alice']

In [36]:
# Removed from the bus... and the hockey team.
hockey_team

['Tina', 'Maya', 'Diana', 'Alice']

In [37]:
class Bus:
    '''The bus model that actually works'''
    
    def __init__(self, passengers=None):
        if passengers is None:
            self.passengers = []
        else:
            # Best practice: make a copy of mutable arguments when possible
            self.passengers = list(passengers)
            
    def pick(self, name):
        self.passengers.append(name)
        
    def drop(self, name):
        self.passengers.remove(name)

In [38]:
hockey_team = ['Sue', 'Tina', 'Maya', 'Diana', 'Pat', 'Alice']
bus6 = Bus(hockey_team)
bus6.passengers

['Sue', 'Tina', 'Maya', 'Diana', 'Pat', 'Alice']

In [39]:
bus6.drop('Sue')
bus6.drop('Pat')
bus6.passengers

['Tina', 'Maya', 'Diana', 'Alice']

In [40]:
hockey_team

['Sue', 'Tina', 'Maya', 'Diana', 'Pat', 'Alice']

-------------
# **Part 3: Data classes**

This section will cover Python features to help avoid boilerplate when creating classes that are essentially collections of fields, similar to a C struct or a database record.

* `collections.namedtuple`
* `typing.NamedTuple`
* `dataclasses.dataclass`

### `collections.namedtuple`

In [41]:
from collections import namedtuple

Coordinate = namedtuple('Coordinate', 'lat long')
cle = Coordinate(41.40, -81.85)
cle

Coordinate(lat=41.4, long=-81.85)

In [42]:
# This syntax feature is called "tuple unpacking"
latitude, longitude = cle
latitude, longitude

(41.4, -81.85)

Includes `__eq__` that knows how to compare with any tuple:

In [43]:
(latitude, longitude) == cle

True

### `namedtuple` limitations

* Instances are ummutable
* No simple way to implement custom methods

### `typing.NamedTuple`

* Added in Python 3.5 
* Variable annotation syntax added in Python 3.6

In [44]:
from typing import NamedTuple, ClassVar

class Coordinate(NamedTuple):
    
    # These are now instance attributes
    lat: float = 0
    long: float = 0
    
    # This is still a class attribute because there's no type annotation
    # This is a very subtle distinction
    reference_system = 'WGS84'
    
    def __str__(self):
        ns = 'NS'[self.lat < 0]
        we = 'EW'[self.long < 0]
        return f'{abs(self.lat):.1f}°{ns}, {abs(self.long):.1f}°{we}'

In [45]:
gulf_of_guinea = Coordinate()
gulf_of_guinea

Coordinate(lat=0, long=0)

In [46]:
cle = Coordinate(41.40, -81.85)
print(cle)

41.4°N, 81.8°W


Assignment (`cle.lat = 3.0`) won't work because we are using tuples, which are immutable.

### @dataclass - `Coordinate` as dataclass

**What is a decorator?** It is a function. When Python reads it, it takes the following class definition and passes it as an object to the decorator, which it can then operate on. Decorators usually create methods, like dataclass for example.

In [47]:
from dataclasses import dataclass
from typing import ClassVar

# This is a decorator
@dataclass
class Coordinate:
    lat: float
    long: float = 0
    
    # This version is an instance attribute:
    # reference_system: str = 'WGS84'
    reference_system: ClassVar[str] = 'WGS84'
        
    def __str__(self):
        ns = 'NS'[self.lat < 0]
        we = 'EW'[self.long < 0]
        return f'{abs(self.lat):.1f}°{ns}, {abs(self.long):.1f}°{we}'

In [48]:
@dataclass?

Signature:
dataclass(
    _cls=None,
    *,
    init=True,
    repr=True,
    eq=True,
    order=False,
    unsafe_hash=False,
    frozen=False,
)
Docstring:
Returns the same class as was passed in, with dunder methods
added based on the fields defined in the class.

Examines PEP 526 __annotations__ to determine fields.

If init is true, an __init__() method is added to the class. If
repr is true, a __repr__() method is added. If order is true, rich
comparison dunder methods are added. If unsafe_hash is true, a
__hash__() method function is added. If frozen is true, fields may
not be assigned to after instance creation.
File:      ~/miniconda3/envs/lambda/lib/python3.7/dataclasses.py
Type:      function


In [ ]:
@dataclass

### A Dublin Core Resource dataclass

In [49]:
from dataclasses import dataclass, field, fields
from typing import List

@dataclass
class Resouce:
    '''Media resouce description'''
    
    # Fields are class attributes that have type annotations
    identifier: str = "0" * 13
    title: str = "<untitled>"
    creators: List[str] = field(default_factory=list)
    date: str = ""
    type: str = ""
    description: str = ""
    language: str = ""
    subjects: List[str] = field(default_factory=list)
    
    def __repr__(self):
        cls = self.__class__
        cls_name = cls.__name__
        res = [f'{cls_name}(']
        for field in fields(cls):
            value = getattr(self, field.name)
            res.append(f'    {field.name} = {value!r},')
        res.append(f')')
        return '\n'.join(res)

In [50]:
description = 'A hands-on guide to idiomatic Python code.'
book = Resouce('9781491946008', 'Fluent Python', ['Luciano Ramalho'],
               '2015-08-20', 'book', description,
               'EN', ['computer programming', 'Python'])
book

Resouce(
    identifier = '9781491946008',
    title = 'Fluent Python',
    creators = ['Luciano Ramalho'],
    date = '2015-08-20',
    type = 'book',
    description = 'A hands-on guide to idiomatic Python code.',
    language = 'EN',
    subjects = ['computer programming', 'Python'],
)

In [51]:
empty = Resouce()
empty

Resouce(
    identifier = '0000000000000',
    title = '<untitled>',
    creators = [],
    date = '',
    type = '',
    description = '',
    language = '',
    subjects = [],
)

In [52]:
# Try `fields?` too
field?

Signature:
field(
    *,
    default=<dataclasses._MISSING_TYPE object at 0x7fdfb4666908>,
    default_factory=<dataclasses._MISSING_TYPE object at 0x7fdfb4666908>,
    init=True,
    repr=True,
    hash=None,
    compare=True,
    metadata=None,
)
Docstring:
Return an object to identify dataclass fields.

default is the default value of the field.  default_factory is a
0-argument function called to initialize a field's value.  If init
is True, the field will be a parameter to the class's __init__()
function.  If repr is True, the field will be included in the
object's repr().  If hash is True, the field will be included in
the object's hash().  If compare is True, the field will be used
in comparison functions.  metadata, if specified, must be a
mapping which is stored but not otherwise examined by dataclass.

It is an error to specify both default and default_factory.
File:      ~/miniconda3/envs/lambda/lib/python3.7/dataclasses.py
Type:      function


------------------------
# **Part 4: Classes**

Let's make two classes `Budget` and `Camper` that can be used to generate a budget report for a group of campers.

In [53]:
import operator

class Camper:
    
    max_name_len = 0
    template = '{name:>{name_len}} paid ${paid:7.2f}'
    
    def __init__(self, name, paid=0.0):
        self.name = name
        self.paid = float(paid)
        if len(name) > Camper.max_name_len:
            Camper.max_name_len = len(name)
    
    def pay(self, amount):
        self.paid += float(amount)
    
    def display(self):
        return Camper.template.format(
            name = self.name,
            name_len = self.max_name_len,
            paid = self.paid,
        )

class Budget:
    '''Represents the budget for a camping trip'''
    
    def __init__(self, *names):
        # Single underscore is a Python convention for declaring a private attribute/method
        # Functionally, there is no difference from naming it without an underscore
        self._campers = {name: Camper(name) for name in names}
    
    def total(self):
        return sum(c.paid for c in self._campers.values())
    
    def people(self):
        return sorted(self._campers)
    
    def contribute(self, name, amount):
        if name not in self._campers:
            raise LookupError("Name not in budget")
        self._campers[name].pay(amount)
    
    def individual_share(self):
        return self.total() / len(self._campers)

    def report(self):
        '''report displays names and amounts due or owed'''
        share = self.individual_share()
        heading_tpl = 'Total: $ {:.2f}; individual share: $ {:.2f}'
        print(heading_tpl.format(self.total(), share))
        print("-"* 42)
        sorted_campers = sorted(self._campers.values(), key=operator.attrgetter('paid'))
        for camper in sorted_campers:
            balance = f'balance: $ {camper.paid - share:7.2f}'
            print(camper.display(), balance, sep=', ')

In [54]:
a = Camper('Anna')
c = Camper('Charlie', 9)

a.pay(33)
a.display()

'   Anna paid $  33.00'

In [55]:
for camper in [a, c]:
     print(camper.display())

   Anna paid $  33.00
Charlie paid $   9.00


In [56]:
b = Budget('Debbie', 'Ann', 'Bob', 'Charlie')
b.total()

0.0

In [57]:
b.people()

['Ann', 'Bob', 'Charlie', 'Debbie']

In [58]:
b.contribute("Bob", 50.00)
b.contribute("Debbie", 40.00)
b.contribute("Ann", 10.00)
b.total()

100.0

In [59]:
b.report()

Total: $ 100.00; individual share: $ 25.00
------------------------------------------
Charlie paid $   0.00, balance: $  -25.00
    Ann paid $  10.00, balance: $  -15.00
 Debbie paid $  40.00, balance: $   15.00
    Bob paid $  50.00, balance: $   25.00


### Sidebar: private attributes

In [60]:
class BlackBox:
    
    def __init__(self, top_content, bottom_content):
        self._top = top_content
        # Two underscores tells Python to "hide" this: make it private
        # See what it does below in b.__dict__
        self.__bottom = bottom_content

b = BlackBox('gold', 'diamonds')
b._top

'gold'

In [61]:
# No __bottom... Where did it go?
hasattr(b, '__bottom')

False

In [62]:
hasattr(b, '_top')

True

In [63]:
# __bottom has been modified 
b.__dict__

{'_top': 'gold', '_BlackBox__bottom': 'diamonds'}

In [64]:
b._BlackBox__bottom

'diamonds'

### Private attribute takeaways

* Its like a safety switch: they won't stop intentional sabotage, but they make reading/writing an attribute more intentional.
* Some Pythonistas use `_private` while others use `__private`
* Its always possible to start with a public attribute, then transform it into a property
* Excessive use of getters/setters is actually weak encapsulation: the class is exposing how it keeps its state

--------------------
# **Part 5: Inheritance**

https://github.com/ramalho/pyob/tree/master/examples/finstory

Proper Python code involving money. `float` errors can affect calculations.

In [65]:
import collections
import decimal

decimal.setcontext(decimal.BasicContext)


def new_decimal(value):
    '''Builds a Decimal using the cleaner float `repr`'''
    if isinstance(value, float):
        value = repr(value)
    return decimal.Decimal(value)


class FinancialHistory:

    def __init__(self, initial_balance=0.0):
        self._balance = new_decimal(initial_balance)
        self._incomes = collections.defaultdict(decimal.Decimal)
        self._expenses = collections.defaultdict(decimal.Decimal)

    def __repr__(self):
        name = self.__class__.__name__
        return f'<{name} balance: {self._balance:.2f}>'

    @property
    def balance(self):
        return self._balance


    def receive(self, amount, source):
        amount = new_decimal(amount)
        self._incomes[source] += amount
        self._balance += amount

    def spend(self, amount, reason):
        amount = new_decimal(amount)
        self._expenses[reason] += amount
        self._balance -= amount

    def received_from(self, source):
        return self._incomes[source]

    def spent_for(self, reason):
        return self._expenses[reason]

### Example of inheritance: 

* requires `super()` 
* Many authors don't recommend inheriting from a concrete class: not a best practice
* "Composition Over Inheritance"

In [66]:
class DeductibleHistory(FinancialHistory):

    def __init__(self, initial_balance=0.0):
        # Delegates initializatin up the chain of inheritance
        super().__init__(initial_balance)
        self._deductions = decimal.Decimal(0)

    def spend(self, amount, reason, deducting=0.0):
        """Record expense with partial deduction"""
        # Uses `spend` from superclass, and add new functionality `deducting`
        super().spend(amount, reason)
        if deducting:
            self._deductions += new_decimal(deducting)

    def spend_deductible(self, amount, reason):
        """Record expense with full deduction"""
        self.spend(amount, reason, amount)

    @property
    def deductions(self):
        return self._deductions

`FinancialHistory`

In [67]:
h = FinancialHistory(100)
h

<FinancialHistory balance: 100.00>

In [68]:
h.spend(39.95, 'meal')
print(f'${h.balance:0.2f}')

$60.05


In [69]:
h.receive(1000.01, "Molly's game")
h.receive(10.00, 'found on street')
h

<FinancialHistory balance: 1070.06>

In [70]:
h.spend(55.36, 'meal')
h.spend(26.65, 'meal')
h.spend(300, 'concert')
h

<FinancialHistory balance: 688.05>

In [71]:
h.spent_for('meal')

Decimal('121.96')

In [72]:
h.spent_for('travel')

Decimal('0')

`DeductibleHistory`

In [73]:
h = DeductibleHistory(1000)
h

<DeductibleHistory balance: 1000.00>

In [74]:
h.spend(600, 'course', 150)
h

<DeductibleHistory balance: 400.00>

In [75]:
h.spend_deductible(250, 'charity')
h

<DeductibleHistory balance: 150.00>

In [76]:
h.spent_for('charity')

Decimal('250')

In [77]:
h.balance

Decimal('150')

In [78]:
h.deductions

Decimal('400')

# **Part 5.1: Composition**

* [Bingo](https://github.com/ramalho/pyob/tree/master/examples/bingo): a simple bingo machine.
* [Tombola ABC](https://github.com/ramalho/pyob/blob/master/examples/tombola/tombola.py): Abstract Base Class for bingo machines.
* [BingoCage](https://github.com/ramalho/pyob/blob/master/examples/tombola/bingo.py): an implementation of Tombola using composition.
* [TumblingDrum](https://github.com/ramalho/pyob/blob/master/examples/tombola/drum.py): another implementation of Tombola using composition.
* [LotteryBlower](https://github.com/ramalho/pyob/blob/master/examples/tombola/lotto.py): yet another implementation of Tombola using composition.
* [Tombolist](https://github.com/ramalho/pyob/blob/master/examples/tombola/tombolist.py): a list subclass registered as a virtual subclass of Tombol

In [79]:
import random


class Bingo:
    def __init__(self, items):
        self._items = list(items)
        random.shuffle(self._items)

    def pop(self):
        return self._items.pop()

    def __len__(self):
        return len(self._items)

In [80]:
# tombola.py
# Example of Abstract Base Class
import abc

class Tombola(abc.ABC):

    @abc.abstractmethod
    def load(self, iterable):
        """Add items from an iterable."""

    @abc.abstractmethod
    def pick(self):
        """Remove item at random, returning it.
        This method should raise `LookupError` when the instance is empty.
        """

    def loaded(self):
        """Return `True` if there's at least 1 item, `False` otherwise."""
        return bool(self.inspect())


    def inspect(self):
        """Return a sorted tuple with the items currently inside."""
        items = []
        while True:
            try:
                items.append(self.pick())
            except LookupError:
                break
        self.load(items)
        return tuple(sorted(items))

In [81]:
# bingo.py
# Example of Composition
class BingoCage(Tombola):

    def __init__(self, items):
        self._randomizer = random.SystemRandom()
        self._items = []
        self.load(items)

    def load(self, items):
        self._items.extend(items)
        self._randomizer.shuffle(self._items)

    def pick(self):
        try:
            return self._items.pop()
        except IndexError:
            raise LookupError('pick from empty BingoCage')

    def __call__(self):
        self.pick()

In [82]:
# drum.py
# Example of Composition
from random import shuffle


class TumblingDrum(Tombola):

    def __init__(self, iterable):
        self._balls = []
        self.load(iterable)

    def load(self, iterable):
        self._balls.extend(iterable)
        shuffle(self._balls)

    def pick(self):
        return self._balls.pop()

In [83]:
# lotto.py
# Example of Composition
import random


class LotteryBlower(Tombola):

    def __init__(self, iterable):
        self._balls = list(iterable)

    def load(self, iterable):
        self._balls.extend(iterable)

    def pick(self):
        try:
            position = random.randrange(len(self._balls))
        except ValueError:
            raise LookupError('pick from empty BingoCage')
        return self._balls.pop(position)

    def loaded(self):
        return bool(self._balls)

    def inspect(self):
        return tuple(sorted(self._balls))

In [84]:
# tombolist.py
# Example of a virtual subclass
from random import randrange

@Tombola.register
class TomboList(list):

    def pick(self):
        if self:
            position = randrange(len(self))
            return self.pop(position)  # <4>
        else:
            raise LookupError('pop from empty TomboList')

    load = list.extend

    def loaded(self):
        return bool(self)

    def inspect(self):
        return tuple(sorted(self))

### Key Takeaways

* Understand the difference between _interface inheritance_ and _implementation inheritance_
* Understand that +interface inheritance_ and _implementation inheritance_ happen at the same time when you subclass a concrete class
* Avoid inheriting from concrete classes
* Beware of non-virtual calls in built-ins. To be safe, subclass `collections.UserList` & co. if you want to roll your own list, dict, etc.
* Favor object composition over class inheritance (Gang of Four)